# Final Report
___
### João Guilherme Almeida

In [ ]:
import subprocess
import time
import random
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
def roda_com_entrada(ex, in_f):
    with open(in_f) as f:
        start = time.perf_counter()
        proc = subprocess.run([ex], input=f.read(), text=True, capture_output=True)
        end = time.perf_counter()
        return proc.stdout, end-start, 

# Introduction

This project aims to work on the DNA sequence pairing problem — that is, finding the similarity between two nucleotide sequences.

For this purpose, several strategies were explored to find the best possible score for the similarity between sequences, namely:

1. Heuristic using Smith-Waterman
2. Local Search with randomness
3. Exhaustive Search

Additionally, the following optimization methods were implemented to improve the exhaustive search performance:
1. OpenMP
2. GPU Parallelization


# Generating Input Files

Multiple input files of different sizes will be generated. This is because, to compare heuristics, it is interesting to submit them to different input sizes to evaluate not only their quality but also execution time, for example.

400 input files were created for the naive heuristics and for the local search; the way to create files for exhaustive search will be different. Also, 5 input files will have both sequences of the same size but with different contents. Finally, to make the second sequence larger, 2 was added to the size of the first.

In [ ]:
for i in range(1, 400, 5): 
    for e in range(0,5):
        n = i
        m = i + 2
        file = "./inputs/dna{0}_{1}.seq".format(i,e)
        f = open(file, 'w')
        seq=[str(n)+'\n',
             str(m)+'\n',
             ''.join(random.choices(['A','T','C','G','-'],k=n))+'\n',
             ''.join(random.choices(['A','T','C','G','-'],k=m))]
        f.writelines(seq)
        f.close()

The test files are structured as follows:
1. The first and second lines of the generated file represent the length of the first and second sequences, respectively
2. The last two lines represent the sequences themselves

# Heuristic with Smith-Waterman

The Smith-Waterman algorithm uses matrices and combinations of sequence characters to deliver an alignment score between them. This score is calculated using match, mismatch, and gap — that is, when the pair of characters (in this project, the nucleotide bases) are equal, different, or when there is a gap, respectively. It is important to note that this algorithm relies heavily on exploitation.

With the generated input files, the tests are started. First, the results and durations of the tests for the naive heuristic using the Smith-Waterman algorithm will be analyzed. For this, a list with the paths of the input files was generated:

In [ ]:
arqs = [f'./inputs/dna{i}_{e}.seq' for i in range(1,400,5) for e in range(0,5)]

For each group of files with the same input size, the group is tested and the execution time for each is recorded.

In [ ]:
qnt_iguais = 4
tempos_entrada_igual = []
tempo_medio = 0
tempos_Heuristica = []
contador = 0
for arq in arqs:
    if(contador < qnt_iguais):
        tempos_entrada_igual.append(roda_com_entrada("./ingenuo/ingenuo", arq))
        contador += 1
    
    else:
        for tempo in tempos_entrada_igual:
            tempo_medio += float(tempo[1])
        tempos_Heuristica.append(tempo_medio/5)
        contador = 0
        tempos_entrada_igual = []

In [ ]:
N_Heuristica = []
qnt_iguais = 4
contador = 0
for arq in arqs:
    if contador < qnt_iguais:
        contador += 1
    else:
        with open(arq) as f:
            n = int(f.readlines()[0])
            N_Heuristica.append(n)
        contador = 0
print(N_Heuristica)


In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
ax.set_xlabel('n')
ax.set_ylabel('time (s)')
ax.scatter(N_Heuristica, tempos_Heuristica)
plt.title("Execution time for various inputs using Smith-Waterman")
plt.show()

It is observed that the naive heuristic execution is very fast, causing all 400 files to be run in very few seconds, even for very large inputs.

___

# Local Search

Local search is an algorithm that focuses more on exploration and less on exploitation. It uses randomness to generate subsequences of different sizes; through these random subsequences the score is calculated and the resulting sequences A and B with the best score are assembled.

For the local search algorithm, it was necessary to increase the step of which files would be used — that is, to decrease the density of entries. Even so, it was possible to run tests for very long inputs, with n reaching almost 400.

In [ ]:
arqs = [f'./inputs/dna{i}_{e}.seq' for i in range(1,400,15) for e in range(0,5)]

In [ ]:
qnt_iguais = 4
tempos_entrada_igual = []
tempo_medio = 0
tempos_local = []
contador = 0
for arq in arqs:
    if(contador < qnt_iguais):
        tempos_entrada_igual.append(roda_com_entrada('./busca_local/busca_local', arq))
        contador += 1
    
    else:
        for tempo in tempos_entrada_igual:
            tempo_medio += float(tempo[1])
        tempos_local.append(tempo_medio/5)
        contador = 0
        tempos_entrada_igual = []

In [ ]:
N_local = []
qnt_iguais = 4
contador = 0
for arq in arqs:
    if contador < qnt_iguais:
        contador += 1
    else:
        with open(arq) as f:
            n = int(f.readlines()[0])
            N_local.append(n)
        contador = 0
print(N_local)


In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
ax.set_xlabel('n')
ax.set_ylabel('time (s)')
ax.scatter(N_local, tempos_local)
plt.title("Execution time for various inputs using Local Search")
plt.show()

When testing local search, it is possible to conclude that it is very time-consuming. This is mainly because it has to split one of the input sequences into smaller parts (using p and k) and test combinations of these parts with the other sequence many times. It was necessary to decrease the number of different splits the algorithm does from 100 to 10 so that the runtime would be feasible.

# Exhaustive Search with Smith-Waterman

Exhaustive search is an algorithm that focuses heavily on exploration; it generates all possible subsequences to find the best combination of the input sequences.

The input files for exhaustive search had to change significantly. First, it was tested locally that runtimes for inputs larger than 80 were extremely high. Thus it was established that the maximum input size would be 70. Additionally, to avoid long runtimes, the number of input files was reduced.

In [ ]:
for i in range(5, 65, 3):
    for e in range (0,5):
        n = i
        m = i + 5
        file = "./inputs_exaust/dna{0}_{1}.seq".format(i,e)# file name to be generated
        f = open(file, 'w')
        seq=[str(n)+'\n',
             str(m)+'\n',
             ''.join(random.choices(['A','T','C','G','-'],k=n))+'\n',
             ''.join(random.choices(['A','T','C','G','-'],k=m))]
        f.writelines(seq)
        f.close()

In [ ]:
arqs = [f'./inputs_exaust/dna{i}_{e}.seq' for i in range(5,65,3) for e in range(0,5)]

In [ ]:
qnt_iguais = 4
tempos_entrada_igual = []
tempo_medio = 0
tempos_exaust = []
contador = 0
for arq in arqs:
    if(contador < qnt_iguais):
        tempos_entrada_igual.append(roda_com_entrada('./busca_exaustiva/busca_exaustiva', arq))
        contador += 1
    
    else:
        for tempo in tempos_entrada_igual:
            tempo_medio += float(tempo[1])
        tempos_exaust.append(tempo_medio/5)
        contador = 0
        tempos_entrada_igual = []

In [ ]:
N_exaust = []
qnt_iguais = 4
contador = 0
for arq in arqs:
    if contador < qnt_iguais:
        contador += 1
    else:
        with open(arq) as f:
            n = int(f.readlines()[0])
            N_exaust.append(n)
        contador = 0
print(N_exaust)


In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
ax.set_xlabel('n')
ax.set_ylabel('time (s)')
ax.scatter(N_exaust, tempos_exaust)
plt.show()

Exhaustive search is very time-consuming because, in addition to what local search does, it breaks both input sequences into all possible subsequences, so it has to do twice the effort.

___

# Exhaustive Search with Truncation

In [ ]:
arqs = [f'./inputs_exaust/dna{i}_{e}.seq' for i in range(5,65,3) for e in range(0,5)]

In [ ]:
qnt_iguais = 4
tempos_entrada_igual = []
tempo_medio = 0
tempos_exaust_trunc = []
contador = 0
for arq in arqs:
    if(contador < qnt_iguais):
        tempos_entrada_igual.append(roda_com_entrada('./busca_exaustiva/busca_exaustiva_trunc', arq))
        contador += 1
    
    else:
        for tempo in tempos_entrada_igual:
            tempo_medio += float(tempo[1])
        tempos_exaust_trunc.append(tempo_medio/5)
        contador = 0
        tempos_entrada_igual = []

In [ ]:
N_exaust_trunc = []
qnt_iguais = 4
contador = 0
for arq in arqs:
    if contador < qnt_iguais:
        contador += 1
    else:
        with open(arq) as f:
            n = int(f.readlines()[0])
            N_exaust_trunc.append(n)
        contador = 0
print(N_exaust_trunc)


In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
ax.set_xlabel('n')
ax.set_ylabel('time (s)')
ax.scatter(N_exaust_trunc, tempos_exaust_trunc)
plt.show()

___

# CPU Parallelism

When testing execution with the non-truncated exhaustive search, it is possible to notice that, as expected, the algorithm achieves the best score. However, achieving this quality is costly, since its runtime is very long. This is mainly because we are treating the problem sequentially — that is, breaking the sequences into all possible subsequences and testing each pair one after another.

Thus, to reduce execution time, we will change the methodology from sequential to parallel. First, we will test CPU parallelization using OpenMP, which leverages multiple CPU threads to execute a specific part of the program.

The initial question in this step would be which part of the code could be parallelized. To answer this, just imagine which part of the code is sequential; it could be two parts: either the generation of subsequences or the score calculation. Ideally, both steps would be parallelized with OpenMP, but we will only parallelize the score calculation and observe how much it improves, recommending that a next iteration would implement OpenMP for subsequence generation as well.

![image.png](attachment:image.png)

To implement this, it was necessary to create a struct composed of a numeric variable "score" and two strings "subsequenceA" and "subsequenceB". Additionally, a vector of this struct was created. Thus, right before computing the score of the subsequences, it was necessary to build this vector with all subsequences and score 0. With the vector built, a parallel for was executed using thread parallelization to compute the subsequence scores in parallel.

In [ ]:
arqs = [f'./inputs_exaust/dna{i}_{e}.seq' for i in range(5,65,3) for e in range(0,5)]

In [ ]:
qnt_iguais = 4
tempos_entrada_igual = []
tempo_medio = 0
tempos_exaust_par_cpu = []
contador = 0
for arq in arqs:
    if(contador < qnt_iguais):
        tempos_entrada_igual.append(roda_com_entrada('./paralelismo_CPU/paralelismo_exaustivo_par', arq))
        contador += 1
    
    else:
        for tempo in tempos_entrada_igual:
            tempo_medio += float(tempo[1])
        tempos_exaust_par_cpu.append(tempo_medio/5)
        contador = 0
        tempos_entrada_igual = []

In [ ]:
N_exaust_par_cpu = []
qnt_iguais = 4
contador = 0
for arq in arqs:
    if contador < qnt_iguais:
        contador += 1
    else:
        with open(arq) as f:
            n = int(f.readlines()[0])
            N_exaust_par_cpu.append(n)
        contador = 0
print(N_exaust_par_cpu)


In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
ax.set_xlabel('n')
ax.set_ylabel('time (s)')
ax.scatter(N_exaust_par_cpu, tempos_exaust_par_cpu)
plt.show()

# Conclusion

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
ax.set_xlabel('n')
ax.set_ylabel('time (s)')
ax.plot(N_Heuristica, tempos_Heuristica, label='Naive Heuristic')
plt.plot(N_local, tempos_local, label='Local Search')
ax.plot(N_exaust, tempos_exaust, label='Exhaustive Search')
plt.plot(N_exaust_trunc, tempos_exaust_trunc, label='Exhaustive Search Trunc')
plt.legend()
plt.show()

From the graph above, it is easy to see that the naive algorithm is the fastest for any input size, and the Exhaustive Search (with the help of the Smith-Waterman algorithm) is extremely slow for large n. The increasing runtime from Naive to Local Search, Exhaustive Search Truncated, and Exhaustive Search is due to the fact that the slower algorithms need to split inputs and perform many additional combinations.

## Comparison of results:

To compare which algorithm achieves better scoring results, two inputs of size 60 were fixed for both the first and second sequence.

GA-C-TCC-GACACTCAGCCTTCAGGC-G-ATGCGCAT-G--TCCGTCGCA-AAGATT-G
GG-AGT-TGGCA-ATG-G-G-CGTTA-ACC--AG-TGCGCTCGGGA-AC-TCCGAGACG-

The obtained results were:

Naive:

Local Search:

Exhaustive Search (Smith-Waterman)

Exhaustive Search with truncation

An interesting point in comparing algorithm scores is that exhaustive search achieved the best performance, but not much better than the naive approach. This raises doubts about the relevance of using exhaustive search, since it is very time-consuming and does not produce a result much better than the naive one. At the same time, more tests would be necessary.

Additionally, something curious was the result of the random local search algorithm that achieved an adequate score, being useful in scenarios to familiarize with the situation and validate a score from another algorithm.

Finally, the exhaustive search with truncation proved very inefficient as it was slow and had very poor scoring performance.

In [ ]:
df = pd.DataFrame(list(zip(tempos_exaust, tempos_exaust_par_cpu)),
               columns =['Non-parallelized', 'Parallelized'])
df

## Profiling

Valgrind was run with the same input above

Naive

![SmithIngenuo1.png](attachment:SmithIngenuo1.png)

![passandomatrix.png](attachment:passandomatrix.png)

Local Search

![busca_local.png](attachment:busca_local.png)

Exhaustive Search (Smith-Waterman)

![busca_exaustiva_Smith_part.png](attachment:busca_exaustiva_Smith_part.png)

![busca_exaustiva.png](attachment:busca_exaustiva.png)

Exhaustive Search Trunc

![Busca_exaustiva_trunc.png](attachment:Busca_exaustiva_trunc.png)

Using Valgrind, it is possible to see that in all algorithms the operations that cause most slowdown are the for-loops. These should be avoided to improve performance. Also, it would be better not to use matrices in some cases, since it is often necessary to traverse them entirely.